In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(stringr)
library(Matrix)
library(R.utils)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

rgeos version: 0.6-3, (SVN revision 696)
 GEOS runtime version: 3.11.0-CAPI-1.17.0 
 Please note that rgeos will be retired during October 2023,
plan transition to sf or terra functions using GEOS at your earliest convenience.
See https://r-spatial.org/r/2023/05/15/evolution4.html for details.
 GEOS using OverlayNG
 Linking to sp version: 1.6-0 
 Polygon checking: TRUE 


Attaching SeuratObject

Attaching sp

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfu

In [2]:
all_dir=list.dirs('/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix')
all_dir=all_dir[grep('[0-9]$',all_dir)]
length(all_dir)
all_dir1=all_dir
all_dir1


[1] 20

[1] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3064-1-230602_web_0"
 [2] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3064-2-230602_web_0"
 [3] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3065-1-230602_web_0"
 [4] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3065-2-230602_web_0"
 [5] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3066-1-230602_web_0"
 [6] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3066-2-230602_web_0"
 [7] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3067-1-230602_web_0"
 [8] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3067-2-230602_web_0"
 [9] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-1-230602_web_0"
[10] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-2-230602_web_0"
[11] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-3-230602_web_0"
[12] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-4-230602_web_0"
[13] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-5-230602_web_0"
[14] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-6-230602_web_0"
[15] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-1-230602_web_0"
[16] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-2-230602_web_0"
[17] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-3-230602_web_0"
[18] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-4-230602_web_0"
[19] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-5-230602_web_0"
[20] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-6-230602_web_0"

In [3]:
for(i in c(1:length(all_dir))){matrix_dir=all_dir[i]

if(file.exists(paste0(matrix_dir,"/barcodes.tsv.gz"))){
    gunzip(paste0(matrix_dir,"/barcodes.tsv.gz"))
    gunzip(paste0(matrix_dir,"/matrix.mtx.gz"))
    if(file.exists(paste0(matrix_dir,"/genes.tsv.gz"))){
        gunzip(paste0(matrix_dir,"/genes.tsv.gz"))}else{
        gunzip(paste0(matrix_dir,"/features.tsv.gz"))
    }
    barcode.path<-paste0(matrix_dir,"/barcodes.tsv")
    if(file.exists(paste0(matrix_dir,"/genes.tsv"))){
        genes.path<-(paste0(matrix_dir,"/genes.tsv"))}else{
        genes.path<-(paste0(matrix_dir,"/features.tsv"))
    }
    matrix.path<-paste0(matrix_dir,"/matrix.mtx")

    zebrafish.data <- readMM(file = matrix.path) ##mac上不能读压缩文件
    gene.names = read.delim(genes.path,header = FALSE, stringsAsFactors = FALSE)
    barcode.names = read.delim(barcode.path,header = FALSE,stringsAsFactors = FALSE)
    colnames(zebrafish.data) = barcode.names$V1
    rownames(zebrafish.data) = gene.names$V1 ##把示例中的V1改成V2

    # check矩阵
    zebrafish.data[1:6, 1:6] ##check矩阵
    dim(zebrafish.data) ##check矩阵

    scRNA <- CreateSeuratObject(counts = zebrafish.data)

    saveRDS(scRNA,paste0(matrix_dir,'/counts.rds'))
}else{
    matrix_dir=paste0(matrix_dir,'/04.Matrix')
    gunzip(paste0(matrix_dir,"/barcodes.tsv.gz"))
    gunzip(paste0(matrix_dir,"/matrix.mtx.gz"))
    if(file.exists(paste0(matrix_dir,"/genes.tsv.gz"))){
        gunzip(paste0(matrix_dir,"/genes.tsv.gz"))}else{
        gunzip(paste0(matrix_dir,"/features.tsv.gz"))
    }

    barcode.path<-paste0(matrix_dir,"/barcodes.tsv")
    if(file.exists(paste0(matrix_dir,"/genes.tsv"))){
        genes.path<-(paste0(matrix_dir,"/genes.tsv"))}else{
        genes.path<-(paste0(matrix_dir,"/features.tsv"))
    }
    matrix.path<-paste0(matrix_dir,"/matrix.mtx")

    zebrafish.data <- readMM(file = matrix.path) ##mac上不能读压缩文件
    gene.names = read.delim(genes.path,header = FALSE, stringsAsFactors = FALSE)
    barcode.names = read.delim(barcode.path,header = FALSE,stringsAsFactors = FALSE)
    colnames(zebrafish.data) = barcode.names$V1
    rownames(zebrafish.data) = gene.names$V1 ##把示例中的V1改成V2

    # check矩阵
    zebrafish.data[1:6, 1:6] ##check矩阵
    dim(zebrafish.data) ##check矩阵

    scRNA <- CreateSeuratObject(counts = zebrafish.data)

    saveRDS(scRNA,paste0(matrix_dir,'/counts.rds'))
}}

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with da

In [4]:
read_file='aba'
for(i in c(1:length(all_dir))){
matrix_dir=all_dir[i]
if(file.exists(paste0(matrix_dir,"/counts.rds"))){
    read_file=c(read_file,(paste0(matrix_dir,"/counts.rds")))
}else{
    matrix_dir=paste0(matrix_dir,'/04.Matrix')
    read_file=c(read_file,(paste0(matrix_dir,"/counts.rds")))
}}
read_file=read_file[-1]
read_file

[1] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3064-1-230602_web_0/04.Matrix/counts.rds"
 [2] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3064-2-230602_web_0/04.Matrix/counts.rds"
 [3] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3065-1-230602_web_0/04.Matrix/counts.rds"
 [4] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3065-2-230602_web_0/04.Matrix/counts.rds"
 [5] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3066-1-230602_web_0/04.Matrix/counts.rds"
 [6] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3066-2-230602_web_0/04.Matrix/counts.rds"
 [7] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3067-1-230602_web_0/04.Matrix/counts.rds"
 [8] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3067-2-230602_web_0/04.Matrix/counts.rds"
 [9] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-1-230602_web_0/04.Matrix/counts.rds"
[10] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-2-230602_web_0/04.Matrix/counts.rds"
[11] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-3-230602_web_0/04.Matrix/counts.rds"
[12] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-4-230602_web_0/04.Matrix/counts.rds"
[13] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-5-230602_web_0/04.Matrix/counts.rds"
[14] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3068-6-230602_web_0/04.Matrix/counts.rds"
[15] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-1-230602_web_0/04.Matrix/counts.rds"
[16] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-2-230602_web_0/04.Matrix/counts.rds"
[17] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-3-230602_web_0/04.Matrix/counts.rds"
[18] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-4-230602_web_0/04.Matrix/counts.rds"
[19] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-5-230602_web_0/04.Matrix/counts.rds"
[20] "/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/3069-6-230602_web_0/04.Matrix/counts.rds"

In [5]:
sceList = lapply(read_file,function(folder){ 
    readRDS(folder)
})
sceList

[[1]]
An object of class Seurat 
19518 features across 7446 samples within 1 assay 
Active assay: RNA (19518 features, 0 variable features)

[[2]]
An object of class Seurat 
19577 features across 7310 samples within 1 assay 
Active assay: RNA (19577 features, 0 variable features)

[[3]]
An object of class Seurat 
19642 features across 6270 samples within 1 assay 
Active assay: RNA (19642 features, 0 variable features)

[[4]]
An object of class Seurat 
19750 features across 6216 samples within 1 assay 
Active assay: RNA (19750 features, 0 variable features)

[[5]]
An object of class Seurat 
19447 features across 7616 samples within 1 assay 
Active assay: RNA (19447 features, 0 variable features)

[[6]]
An object of class Seurat 
19557 features across 6869 samples within 1 assay 
Active assay: RNA (19557 features, 0 variable features)

[[7]]
An object of class Seurat 
19560 features across 5619 samples within 1 assay 
Active assay: RNA (19560 features, 0 variable features)

[[8]]
An obje

In [6]:
read_file_renew=str_extract(read_file,'[0-9][0-9][0-9][0-9]-[0-9]*')

read_file_renew


[1] "3064-1" "3064-2" "3065-1" "3065-2" "3066-1" "3066-2" "3067-1" "3067-2"
 [9] "3068-1" "3068-2" "3068-3" "3068-4" "3068-5" "3068-6" "3069-1" "3069-2"
[17] "3069-3" "3069-4" "3069-5" "3069-6"

In [7]:
unique(read_file_renew)

[1] "3064-1" "3064-2" "3065-1" "3065-2" "3066-1" "3066-2" "3067-1" "3067-2"
 [9] "3068-1" "3068-2" "3068-3" "3068-4" "3068-5" "3068-6" "3069-1" "3069-2"
[17] "3069-3" "3069-4" "3069-5" "3069-6"

In [8]:

sce.big <- merge(sceList[[1]], 
                 y = c(sceList[[2]],sceList[[3]],sceList[[4]],
                       sceList[[5]],sceList[[6]],
                       sceList[[7]],sceList[[8]],sceList[[9]],sceList[[10]],sceList[[11]],
                       sceList[[12]],sceList[[13]],
                       sceList[[14]], sceList[[15]],sceList[[16]],sceList[[17]],sceList[[18]],sceList[[19]],sceList[[20]]), 
                 add.cell.ids = read_file_renew,
                 project = "macaca")

table(sce.big$orig.ident)

sce.big@meta.data$group=str_extract(rownames(sce.big@meta.data),'[0-9][0-9][0-9][0-9]-[0-9]')




SeuratProject 
       129748 

In [10]:
length(unique(sce.big@meta.data$group))

[1] 20

In [11]:
saveRDS(sce.big,'/home/liuyx/liuyuxuan/spa_seurat/cla/macaca/sn_0709_renew/soupX.flt_Matrix/macaque_snrna_all_soupX_unfil_batch2_0709.rds')
